# **Auto-Correct Algorithm**
 **We’ll implement the four steps of the auto-correct algorithm:**

1. **Check if a word is misspelled.**
2. **Generate words within a certain edit distance.**
3. **Filter valid candidates from vocabulary.**
4. **Find the most probable candidate.**

In [79]:
import re
from collections import Counter

In [80]:
corpus = "cat bat cat mat cat rat bat cat" # Corpus: dummy to auto correct word "cat"
vocabulary = set(corpus.split()) # Vocabulary: Unique values from corpus
vocabulary

{'bat', 'cat', 'mat', 'rat'}

In [81]:
word_counts = Counter(corpus.split()) # Word Count of Each word in corpus
total_words = sum(word_counts.values()) # Total Number of Words
total_words

8

In [82]:
# Below in the loop the variable is taking word and its count form the variable total_words and calculating probablity of each word using:
# P(W) = Word / sum of all count
# word = {'bat', 'cat', 'mat', 'rat'}, count = {1, 2, 4}

word_probalities = {word: count/total_words for word, count in word_counts.items()}

In [83]:
# Step 1: Check if a word is misspelled, means not in vocabulary of a particular word(cat)
def is_misspelled(word):
    return word not in vocabulary

In [84]:
# Step 2: Generate words within 1 edit distance
def edit_distance_one(word):
    letters = 'abcdefghijklmnopqrstuvwxyz'
    splits = [(word[:i], word[i:]) for i in range(len(word)+1)]
    
    inserts = [L + c + R for L, R in splits for c in letters]
    deletes = [L + R[1:] for L, R in splits if R]
    substitutes = [L + c + R[1:] for L, R in splits if R for c in letters]
    swaps = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R) > 1]
    
    return set(inserts + deletes + substitutes + swaps)

In [85]:
# Step 3: Filter valid candidates
def valid_candidates(candidates):
    return [word for word in candidates if word in vocabulary]

In [86]:
# Step 4: Choose the most probable word
def most_probable_word(candidates):
    if not candidates:
        return None
    return max(candidates, key=lambda word: word_probalities.get(word, 0))

In [87]:
# Full auto-correct function
def auto_correct(word):
    if not is_misspelled(word):
        return word  # Already correct
    candidates = edit_distance_one(word)
    valid_words = valid_candidates(candidates)
    return most_probable_word(valid_words) or word  # Return original word if no valid candidates

In [88]:
misspelled_word = "cet"
corrected_word = auto_correct(misspelled_word)
print(f"Original: {misspelled_word}, Corrected: {corrected_word}")

Original: cet, Corrected: cat


## Libraries for Auto-Correct

In [89]:
from textblob import Word

word = Word("cet")
corrected_word = word.correct()
print(f"Corrected: {corrected_word}")

Corrected: cet


In [90]:
from spellchecker import SpellChecker

spell = SpellChecker()
misspelled_word = "caet"
corrected_word = spell.correction(misspelled_word)
print(f"Corrected: {corrected_word}")

Corrected: cat


- Find Candidates:

In [91]:
candidates = spell.candidates(misspelled_word)
print(f"Candidates: {candidates}")

Candidates: {'cast', 'cadet', 'caen', 'cart', 'cate', 'cat', 'capt', 'capet', 'cant', 'caret', 'catt'}


In [92]:
from fuzzywuzzy import process

word = "cst"
choices = ["cat", "bat", "mat", "rat"]
best_match = process.extractOne(word, choices)
print(f"Best match: {best_match[0]}")

Best match: cat


# Minimum Edit Distance

In [93]:
source_word = "cat"
target_word = "bat"
rows, cols = len(source_word) + 1, len(target_word) + 1
dp = [[0] * cols for _ in range(rows)]
dp

[[0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0], [0, 0, 0, 0]]

In [94]:
for i in range(rows):
    dp[i][0] = i
for j in range(cols):
    dp[0][j] = j


In [72]:
dp

[[0, 1, 2, 3], [1, 0, 0, 0], [2, 0, 0, 0], [3, 0, 0, 0]]

In [95]:
def minimum_edit_distance(source, target):
    # Create DP table
    rows, cols = len(source) + 1, len(target) + 1
    dp = [[0] * cols for _ in range(rows)]

    # Initialize base cases
    for i in range(rows):
        dp[i][0] = i  # Cost of deleting all characters
    for j in range(cols):
        dp[0][j] = j  # Cost of inserting all characters

    # Fill the DP table
    for i in range(1, rows):
        for j in range(1, cols):
            if source[i - 1] == target[j - 1]:  # Characters match
                dp[i][j] = dp[i - 1][j - 1]  # No cost
            else:
                insert = dp[i][j - 1] + 1
                delete = dp[i - 1][j] + 1
                substitute = dp[i - 1][j - 1] + 1
                dp[i][j] = min(insert, delete, substitute)

    # Final result is in the bottom-right cell
    return dp[-1][-1]

In [96]:
# Test
source_word = "cat"
target_word = "bat"
edit_distance = minimum_edit_distance(source_word, target_word)
print(f"Minimum edit distance between '{source_word}' and '{target_word}': {edit_distance}")

Minimum edit distance between 'cat' and 'bat': 1


## Libraries for Minimum Edit Distance

### 1. nltk
Description: A robust library for NLP with capabilities for stemming, lemmatization, and edit distance.
Features:
- Includes an implementation of Levenshtein distance for minimum edit distance calculations.


In [97]:
import nltk
from nltk.metrics import edit_distance

source_word = "cat"
target_word = "bat"
distance = edit_distance(source_word, target_word)
print(f"Edit Distance: {distance}")

Edit Distance: 1
